In [ ]:
#1. Basic Neural Network (Feedforward / MLP)
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models

model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[4,]),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='softmax'),
])
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
#2. Convolutional Neural Network (CNN)
model=tf.keras.Sequential([
    layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='softmax'),
])
#history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

In [ ]:
#Early Stopping & Model Checkpointing

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True),
]

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
lr=ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.0001)

In [ ]:
from tensorflow.keras.applications import ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
)
train_data = train_datagen.flow_from_directory(
    "data/train", target_size=(224, 224), batch_size=32, shuffle=True, seed=42, class_mode='categorical',
    subset='training'
)
val_data = train_datagen.flow_from_directory(
    "data/val", target_size=(224, 224), batch_size=32, shuffle=True, seed=42, class_mode='categorical',
    subset='validation'
)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(train_data.num_classes, activation='softmax'),
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.0001)
]
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    callbacks=callbacks
)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb

vocab_size = 10000
max_length = 200
(X_train, y_train), (X_val, y_val) = imdb.load_data(num_words=vocab_size)
X_train = pad_sequences(X_train, maxlen=max_length)
X_val = pad_sequences(X_val, maxlen=max_length)

model=models.Sequential([
    layers.Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length),
    layers.SpatialDropout1D(0.2),
    layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    layers.Dense(64, activation='sigmoid')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.0001)
]

#history =model.fit(X_train, y_train, epochs=10, batch_size=32, callbacks=callbacks)

In [8]:
vocab_size = 10000
max_length = 200

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)
X_train = pad_sequences(X_train, maxlen=max_length)
X_test = pad_sequences(X_test, maxlen=max_length)


def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    #Self-attention
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)

    #feedforward
    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(inputs.shape[-1])(ff)
    x = layers.Dropout(dropout)(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x+ff)
    return x


inputs = layers.Input(shape=(max_length,))
x = layers.Embedding(input_dim=vocab_size, output_dim=64)(inputs)
positions = tf.range(start=0, limit=max_length, delta=1)
pos_embedding_layer = layers.Embedding(input_dim=max_length, output_dim=64)(positions)
x = x + pos_embedding_layer

x = transformer_encoder(x, head_size=64, num_heads=16, ff_dim=64, dropout=0.2)
x = transformer_encoder(x, head_size=64, num_heads=16, ff_dim=64, dropout=0.2)

x = layers.GlobalMaxPool1D()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = models.Model(inputs, outputs)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 200)]                0         []                            
                                                                                                  
 embedding_9 (Embedding)     (None, 200, 64)              640000    ['input_5[0][0]']             
                                                                                                  
 tf.__operators__.add_12 (T  (None, 200, 64)              0         ['embedding_9[0][0]']         
 FOpLambda)                                                                                       
                                                                                                  
 multi_head_attention_7 (Mu  (None, 200, 64)              265280    ['tf.__operators__.add_1

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, head_size, num_heads, ff_dim, dropout=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads)
        self.ffn = models.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(head_size * num_heads),
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(dropout)
        self.dropout2 = layers.Dropout(dropout)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        out1 = self.layernorm1(inputs + self.dropout1(attn_output, training=training))
        ffn_output = self.ffn(out1)
        return self.layernorm2(out1 + self.dropout2(ffn_output, training=training))

# Step 4: Build the Model
def build_model():
    inputs = layers.Input(shape=(max_length,))
    x = layers.Embedding(input_dim=vocab_size, output_dim=64)(inputs)

    # Positional Encoding
    positions = tf.range(start=0, limit=max_length, delta=1)
    pos_embedding = layers.Embedding(input_dim=max_length, output_dim=64)(positions)
    x = x + pos_embedding

    # Transformer Blocks
    x = TransformerBlock(head_size=64, num_heads=2, ff_dim=64, dropout=0.1)(x)
    x = TransformerBlock(head_size=64, num_heads=2, ff_dim=64, dropout=0.1)(x)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = models.Model(inputs, outputs)
    return model

# Step 5: Compile Model
model = build_model()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])



In [ ]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from transformers import DataCollatorWithPadding
from datasets import load_dataset
import tensorflow as tf

dataset = load_dataset("imdb")
dataset = dataset.shuffle(seed=42)
train_data = dataset["train"].select(range(10000))  # Smaller for speed
test_data = dataset["test"].select(range(5000))

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding=True)

train_data = train_data.map(tokenize, batched=True)
test_data = test_data.map(tokenize, batched=True)

# Step 5: Convert to tf.data.Dataset
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train = train_data.to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["label"],
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator
)

tf_test = test_data.to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["label"],
    shuffle=False,
    batch_size=32,
    collate_fn=data_collator
)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

model.fit(tf_train, validation_data=tf_test, epochs=3)

# Step 8: Evaluate
loss, acc = model.evaluate(tf_test)
print(f"Test Accuracy: {acc:.4f}")
